# Neighborly Notebook Sample

In [2]:
import random
import time

import networkx as nx

from neighborly.core.life_event import LifeEventLog
from neighborly.core.time import SimDateTime
from neighborly.exporter import NeighborlyJsonExporter
from neighborly.plugins.default_plugin import DefaultPlugin
from neighborly.plugins.talktown import TalkOfTheTownPlugin
from neighborly.plugins.weather_plugin import WeatherPlugin
from neighborly.simulation import Simulation

In [3]:
sim = (
    Simulation.create(
        seed=random.randint(0, 999999),
        world_gen_start=SimDateTime(year=1929, month=8, day=19),
        world_gen_end=SimDateTime(year=1979, month=8, day=19),
    )
    .add_plugin(DefaultPlugin())
    .add_plugin(WeatherPlugin())
    .add_plugin(TalkOfTheTownPlugin())
)

In [4]:
sim.world.get_resource(LifeEventLog).subscribe(lambda e: print(f"{str(e)}"))
sim.establish_setting()

NewResidenceBuilt [at 1929-08-19T12:00.000z] : Residence:1
NewBusinessBuilt [at 1929-08-19T12:00.000z] : Business:2
MoveIntoTown [at 1929-08-26T12:00.000z] : resident:3, resident:4, resident:5, resident:6, resident:7
BecomeAdult [at 1929-08-26T12:00.000z] : Character:3
BecomeAdult [at 1929-08-26T12:00.000z] : Character:4
BecomeTeen [at 1929-08-26T12:00.000z] : Character:5
BecomeTeen [at 1929-08-26T12:00.000z] : Character:6
BecomeTeen [at 1929-08-26T12:00.000z] : Character:7
BecameBusinessOwner [at 1929-08-26T12:00.000z] : Business:2, Owner:4
HiredAtBusiness [at 1929-08-26T12:00.000z] : Business:2, Employee:3
BecomeYoungAdult [at 1929-08-27T00:00.000z] : Character:5
BecomeYoungAdult [at 1929-08-27T00:00.000z] : Character:6
BecomeYoungAdult [at 1929-08-27T00:00.000z] : Character:7
HiredAtBusiness [at 1929-08-27T00:00.000z] : Business:2, Employee:6
BecomeAdult [at 1929-08-27T12:00.000z] : Character:5
BecomeAdult [at 1929-08-27T12:00.000z] : Character:7
HiredAtBusiness [at 1929-08-27T12:00

BecomeAdult [at 1931-10-26T12:00.000z] : Character:30
GotPregnant [at 1932-03-19T12:00.000z] : PersonA:25, PersonB:26
ChildBirth [at 1933-00-19T12:00.000z] : Parent:25, Parent:26, Child:61
GotPregnant [at 1933-10-19T12:00.000z] : PersonA:9, PersonB:10
ChildBirth [at 1934-07-19T12:00.000z] : Parent:9, Parent:10, Child:62
BecomeElder [at 1933-10-26T00:00.000z] : Character:29
GotPregnant [at 1934-00-19T12:00.000z] : PersonA:55, PersonB:56
ChildBirth [at 1934-09-19T12:00.000z] : Parent:55, Parent:56, Child:63
BecomeElder [at 1934-01-19T00:00.000z] : Character:60
BecomeAdult [at 1934-01-19T12:00.000z] : Character:56
GotPregnant [at 1934-02-19T12:00.000z] : PersonA:4, PersonB:3
ChildBirth [at 1934-11-19T12:00.000z] : Parent:4, Parent:3, Child:64
Retire [at 1934-03-19T12:00.000z] : Retiree:60
GotPregnant [at 1934-09-19T12:00.000z] : PersonA:10, PersonB:9
ChildBirth [at 1935-06-19T12:00.000z] : Parent:10, Parent:9, Child:65
BecomeElder [at 1935-00-19T00:00.000z] : Character:42
BecomeAdult [at 

StartDating [at 1967-10-19T12:00.000z] : PersonA:10, PersonB:9
BecomeAdult [at 1968-00-26T12:00.000z] : Character:83
GotMarried [at 1968-01-19T12:00.000z] : PersonA:10, PersonB:9
GotMarried [at 1968-03-19T12:00.000z] : PersonA:25, PersonB:53
GotMarried [at 1968-04-19T12:00.000z] : PersonA:26, PersonB:54
GotPregnant [at 1968-11-19T12:00.000z] : PersonA:15, PersonB:14
ChildBirth [at 1969-08-19T12:00.000z] : Parent:15, Parent:14, Child:94
GotPregnant [at 1969-00-19T12:00.000z] : PersonA:48, PersonB:49
ChildBirth [at 1969-09-19T12:00.000z] : Parent:48, Parent:49, Child:95
BecomeElder [at 1969-01-26T00:00.000z] : Character:93
Retire [at 1969-02-19T12:00.000z] : Retiree:93
Death [at 1969-08-26T00:00.000z] : Character:4
StartDating [at 1970-00-19T12:00.000z] : PersonA:4, PersonB:3
GotMarried [at 1970-00-19T12:00.000z] : PersonA:4, PersonB:3
Death [at 1970-01-19T00:00.000z] : Character:59
GotPregnant [at 1970-05-19T12:00.000z] : PersonA:53, PersonB:25
ChildBirth [at 1971-02-19T12:00.000z] : Pa

In [153]:
from neighborly.core.relationship import RelationshipGraph, RelationshipTag
import matplotlib.pyplot as plt
from pyvis.network import Network
from neighborly.core.character import GameCharacter
from neighborly.core.residence import Resident
import random


rel_graph = sim.world.get_resource(RelationshipGraph)


# G = nx.MultiGraph()
G = Network(width=800, height=800, notebook=True, bgcolor='#ffffff', directed=True)

# Randomly select a character present in the town
start_id = random.choice(sim.world.get_components(GameCharacter, Resident))[0]
# start_id = 67

# Starting from that character id, perform a breadth first search
# up to a given depth
depth_limit = 1

# Queue of nodes to visit
queue = [(0, start_id)]
 
# Nodes that have been visited during this search
visited_nodes = set()

while queue:
    depth, character_id = queue.pop(0)
    character = sim.world.get_gameobject(character_id).get_component(GameCharacter)
    
    
    # Insert a node for this character
    visited_nodes.add(character_id)
    G.add_node(
        character_id, 
        label=str(character.name),
        color='blue',
        size=10
    )
    
    if depth >= depth_limit:
        continue
    
    # Insert edges for each relationship this character has
    for relationship in rel_graph.get_relationships(character_id):
        other_character = sim.world.get_gameobject(relationship.target)
        if other_character.has_component(Resident):
            G.add_node(
                relationship.target, 
                label=str(other_character.get_component(GameCharacter).name),
                color='blue',
                size=10
            )

            G.add_edge(
                    relationship.owner, 
                    relationship.target, 
                    color='green', 
                    label=str(int(relationship.friendship)),

            )
            
            G.add_edge(
                    relationship.owner, 
                    relationship.target, 
                    color='red',
                    label=str(int(relationship.romance)),
            )

            if relationship.target not in visited_nodes:
                queue.append((depth + 1, relationship.target))
        

# for gid, (character, _) in sim.world.get_components(GameCharacter, Resident):
    
#     if gid not in created_nodes:
#         created_nodes.add(gid)
#         G.add_node(
#             gid, 
#             label=character.name.firstname,
#             color='red',
#             size=35
#         )
#     for rel in rel_graph.get_relationships(gid):
#         if sim.world.get_gameobject(rel.target).has_component(Resident) and rel.friendship >= 30:
# #         if rel.has_tags(RelationshipTag.Parent):
#             G.add_edge(
#                 rel.owner, 
#                 rel.target, 
#                 color='black', 
#                 weight=abs(rel.friendship // 15)
                
#             )
            
#             if rel.target not in created_nodes:
#                 created_nodes.add(rel.target)
#                 G.add_node(
#                     rel.target, 
#                     label=(
#                         sim.world.get_gameobject(rel.target)
#                         .get_component(GameCharacter)
#                         .name
#                         .firstname
#                     ),
#                     color='red',
#                     size=35
#                 )

            
    

# g = Network(width=800, height=800, notebook=True, bgcolor='#ffffff')
# g.from_nx(G)
G.force_atlas_2based()
G.show("sample.html")

# nx.shell_layout(G)
# ax = plt.subplot()
# nx.draw(G)

# widths = nx.get_edge_attributes(G, 'weight')
# nodelist = G.nodes()

# plt.figure(figsize=(12,8))

# pos = nx.shell_layout(G)
# nx.draw_networkx_nodes(G,pos,
#                        nodelist=nodelist,
#                        node_size=1500,
#                        node_color='black',
#                        alpha=0.7)
# nx.draw_networkx_edges(G,pos,
#                        edgelist = widths.keys(),
#                        width=list(widths.values()),
#                        edge_color='lightblue',
#                        alpha=0.6)
# nx.draw_networkx_labels(G, pos=pos,
#                         labels=dict(zip(nodelist,nodelist)),
#                         font_color='white')
# plt.box(False)
# plt.show()